In [ ]:
import os
import json
import time
import copy
from datetime import datetime
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
import lovely_tensors as lt # can be removed
import numpy as np
import dill
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from fl2o.optimizee import MLPOptee, CustomParams
from fl2o.optimizee_modules import MetaParameter
from fl2o.optimizer import GD, Adam, FGD, AFOGD, CFGD, CFGD_ClosedForm, L2O_Update
from fl2o.l2o import L2O
from fl2o.data import MNIST, CustomTask, generate_least_squares_task, H1, H2, H3
from fl2o.training import do_fit, find_best_lr, meta_train, get_optimal_lr, n_step_lookahead_lr_search_hfunc_tanh_twolayer_optee
from fl2o.utils import dict_to_str, plot_log, plotter, plot_metrics, apply_publication_plt_settings, plot_strategy

lt.monkey_patch() # can be removed

DATA_PATH = os.getenv("DATA_PATH")
CKPT_PATH = os.getenv("CKPT_PATH")
DEVICE = os.getenv("DEVICE", "cpu")

print(f"{DATA_PATH=}\n{CKPT_PATH=}\n{DEVICE=}")

In [ ]:
### load previous checkpoint of l2o-cfgd
ckpt = torch.load(
    os.path.join(
        CKPT_PATH,
        "l2o",
        "26-07_19-18__L2O__CFGD",
        "ckpt_1200.pt"
    ),
    map_location=torch.device(DEVICE),
    pickle_module=dill,
)
config = ckpt["config"]
config["device"] = DEVICE
print(json.dumps(config, indent=4, default=str))

l2o_dict = ckpt["l2o_dict"]
l2o_dict_best = ckpt["l2o_dict_best"]
if "opter" not in l2o_dict_best:
    l2o_dict_best["best_l2o_dict"]["opter"] = config["opter"]["opter_cls"](**config["opter"]["opter_config"])
    l2o_dict_best["best_l2o_dict"]["opter"].load_state_dict(l2o_dict_best["best_l2o_dict"]["opter_state_dict"])
    l2o_dict_best["best_l2o_dict"]["meta_opter"] = config["meta_training_config"]["meta_opter_cls"](
        params=l2o_dict_best["best_l2o_dict"]["opter"].parameters(),
        **config["meta_training_config"]["meta_opter_config"],
    )
    l2o_dict_best["best_l2o_dict"]["meta_opter"].load_state_dict(l2o_dict_best["best_l2o_dict"]["meta_opter_state_dict"])
l2o_dict_best["best_l2o_dict"]["opter"].device = DEVICE
l2o_dict_best["best_l2o_dict"]["opter"].to(DEVICE)
l2o_dict["opter"].device = DEVICE
l2o_dict["opter"].to(DEVICE)
log = ckpt["log"]
print(json.dumps(config, indent=4, default=str))

In [ ]:
### load previous checkpoint of l2o
ckpt_2 = torch.load(
    os.path.join(
        CKPT_PATH,
        "l2o",
        "26-07_19-21__L2O__L2O_Update",
        "ckpt_1200.pt"
    ),
    map_location=torch.device(DEVICE),
    pickle_module=dill,
)
l2o_dict_2 = ckpt_2["l2o_dict"]
l2o_dict_best_2 = ckpt_2["l2o_dict_best"]
config_2 = ckpt_2["config"]
if "opter" not in l2o_dict_best_2:
    l2o_dict_best_2["best_l2o_dict"]["opter"] = config_2["opter"]["opter_cls"](**config_2["opter"]["opter_config"])
    l2o_dict_best_2["best_l2o_dict"]["opter"].load_state_dict(l2o_dict_best_2["best_l2o_dict"]["opter_state_dict"])
    l2o_dict_best_2["best_l2o_dict"]["meta_opter"] = config_2["meta_training_config"]["meta_opter_cls"](
        params=l2o_dict_best_2["best_l2o_dict"]["opter"].parameters(),
        **config_2["meta_training_config"]["meta_opter_config"],
    )
    l2o_dict_best_2["best_l2o_dict"]["meta_opter"].load_state_dict(l2o_dict_best_2["best_l2o_dict"]["meta_opter_state_dict"])
l2o_dict_best_2["best_l2o_dict"]["opter"].device = DEVICE
l2o_dict_best_2["best_l2o_dict"]["opter"].to(DEVICE)
l2o_dict_2["opter"].device = DEVICE
l2o_dict_2["opter"].to(DEVICE)
log_2 = ckpt_2["log"]
print(json.dumps(config_2, indent=4, default=str))

## MNIST

### Meta-training

In [ ]:
config = {
    "time": datetime.now().strftime("%d-%m_%H-%M"),
}

### data (task)
config["data"] = {
    "data_cls": MNIST,
}
config["data"]["data_config"] = {
    "device": DEVICE,
    "preload": True,
}

### optimizee
config["optee"] = {
    "optee_cls": MLPOptee,
    "optee_config": {
        "layer_sizes": [20],
        "act_fn": nn.ReLU(),
        "device": DEVICE,
    },
}

### optimizer L2O-CFGD
config["opter"] = {
    "opter_cls": L2O,
    "opter_config": {
        "in_dim": 3, # len(in_features) + 1
        "out_dim": 3,
        "hidden_sz": 40,
        "in_features": ("grad", "iter_num_enc"),
        "device": DEVICE,
        "base_opter_cls": CFGD,
        "base_opter_config": {
            "lr": 0.1,
            "alpha": None,
            "beta": None,
            "c": None,
            "s": 1,
            "version": "NA",
            "init_points": None,
            "detach_gauss_jacobi": True,
            "device": DEVICE,
        },
        "params_to_optimize": {
            "alpha": {
                "idx": 0,
                "act_fns": ("sigmoid",),
            },
            "beta": {
                "idx": 1,
                "act_fns": ("identity",),
            },
            "c": {
                "idx": 2,
                "act_fns": ("identity",),
            },
        },
    },
}

### optimizer L2O
# config["opter"] = {
#     "opter_cls": L2O,
#     "opter_config": {
#         "in_dim": 3, # len(in_features) + 1
#         "out_dim": 1,
#         "hidden_sz": 40,
#         "in_features": ("grad", "iter_num_enc"),
#         "base_opter_cls": L2O_Update,
#         "device": DEVICE,
#         "base_opter_config": {
#             "lr": 0.1,
#             "device": DEVICE,
#         },
#         "params_to_optimize": {
#             "update": {
#                 "idx": 0,
#                 "act_fns": ("identity",),
#             },
#         },
#     },
# }

### meta-training config
config["meta_training_config"] = {
    "meta_opter_cls": optim.Adam,
    "meta_opter_config": {
        "lr": 1e-3,
    },
    "n_runs": 600,
    "unroll": 40,
    "loggers": [
        # {
        #     "every_nth_run": 20,
        #     "logger_fn": partial(plotter, to_plot="c"),
        # }
    ],
}

### other
config.update({
    "n_iters": 400,
    "l2o_dict": None,
    "additional_metrics": {
    },
    "ckpt_config": {
        "ckpt_every_nth_run": 20,
        "ckpt_dir": os.path.join(
            CKPT_PATH,
            "l2o",
            config["time"] + "__"\
                + config["opter"]["opter_cls"].__name__ + "__"\
                + config["opter"]["opter_config"]["base_opter_cls"].__name__,
        ),
    },
    "device": DEVICE,
    "seed": 0,
})
config["ckpt_config"]["ckpt_dir_meta_training"] = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    "meta_training",
)
config["ckpt_config"]["ckpt_dir_meta_testing"] = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    "meta_testing",
)

### make dirs
os.makedirs(config["ckpt_config"]["ckpt_dir"], exist_ok=True)
os.makedirs(config["ckpt_config"]["ckpt_dir_meta_training"], exist_ok=True)
os.makedirs(config["ckpt_config"]["ckpt_dir_meta_testing"], exist_ok=True)

### save config
with open(os.path.join(config["ckpt_config"]["ckpt_dir"], "config.json"), "w") as f:
    json.dump(config, f, indent=4, default=str)

print(f"Path to checkpoints: {config['ckpt_config']['ckpt_dir']}")
print(os.path.basename(config["ckpt_config"]["ckpt_dir"]))

In [ ]:
### meta train
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])
l2o_dict, l2o_dict_best, log = meta_train(
    config=config,
    device=config["device"],
    ### uncomment below to keep meta-training
    # l2o_dict=l2o_dict,
    # l2o_dict_best=l2o_dict_best,
    # log=log,
)

### save checkpoint
torch.save({
    "l2o_dict": l2o_dict,
    "l2o_dict_best": l2o_dict_best,
    "log": log,
    "config": config,
}, os.path.join(config["ckpt_config"]["ckpt_dir"], "ckpt.pt"), pickle_module=dill)

plt.plot(log["loss_sum"])

### Meta-testing

##### Config

In [ ]:
n_test_runs = 1
test_run_iters = 500
test_runs_seed = 1

update_config_base = {"n_iters": test_run_iters}
update_config_base["optee"] = {
    "optee_cls": MLPOptee,
    "optee_config": {
        "layer_sizes": [20],
        "act_fn": nn.ReLU(),
        "device": DEVICE,
    },
}
update_config_base["additional_metrics"] = {}

runs = dict()

In [ ]:
### GD
runs["GD"] = {
    "update_config": {
        **update_config_base,
        "opter": {
            "opter_cls": GD,
            "opter_config": {
                "lr": 0.3,
                "device": DEVICE,
            },
        },
    },
    "plot_config": {
        "color": "black",
        "linestyle": "dashed",
    },
    # "load_saved": False,
}

In [ ]:
### NA-CFGD
# hyperparam search --->
# for lr in [0.04, 0.02, 0.003]:
#     for alpha in [0.3, 0.6, 0.9, 0.95]:
#         for beta in [-20, -2, 0, 2, 20]:
#             for c in [-1, 0, 1]:
# <--- hyperparam search
for lr in [0.02]:
    for alpha in [0.1]:
        for beta in [0]:
            for c in [0]:
                runs[r"NA-CFGD, $\alpha$=" + str(alpha)
                    + r" $\beta$=" + str(beta)
                    + r" $c$=" + str(c)
                    + r" $\eta$=" + str(lr)] = {
                    "update_config": {
                        **update_config_base,
                        "opter": {
                            "opter_cls": CFGD,
                            "opter_config": {
                                "lr": lr,
                                "alpha": alpha,
                                "beta": beta,
                                "c": torch.tensor([c], device=DEVICE).float(),
                                "s": 1,
                                "n_hutchinson_steps": 5,
                                "version": "NA",
                                "init_points": None,
                                "device": DEVICE,
                            },
                        },
                    },
                    "plot_config": {
                        "linestyle": "dashed",
                    },
                    "load_saved": False,
                }

In [ ]:
### AT-CFGD
_tmp_optee = update_config_base["optee"]["optee_cls"](**update_config_base["optee"]["optee_config"])

# hyperparam search --->
for L in [1, 2, 3, 4]:
    for alpha in [0.2, 0.6, 0.9]:
        for beta in [-10, -2, -1, 0, 1, 2, 10]:
            for lr in [0.1, 0.5, 1]:
# <--- hyperparam search

# for L in [1]:
#     for lr in [0.1]:
#         for alpha in [0.2]:
#             for beta in [-1]:
                runs[r"AT-CFGD, $\L$=" + str(L)
                    + r" $\alpha$=" + str(alpha)
                    + r" $\beta$=" + str(beta)
                    + r" $\eta$=" + str(lr)] = {
                    "update_config": {
                        **update_config_base,
                        "opter": {
                            "opter_cls": CFGD,
                            "opter_config": {
                                "lr": lr,
                                "alpha": alpha,
                                "beta": beta,
                                "c": None,
                                "s": 1,
                                "version": "AT",
                                "init_points": [
                                    [torch.randn_like(p, requires_grad=False, device=DEVICE) for _ in range(L)] for _, p in _tmp_optee.all_named_parameters()
                                ],
                                "device": DEVICE,
                            },
                        },
                    },
                    "plot_config": {
                        "linestyle": "dashed",
                    },
                    # "load_saved": False,
                }

In [ ]:
### L2O
runs["L2O"] = {
    "update_config": {
        "n_iters": update_config_base["n_iters"],
        "optee": update_config_base["optee"],
        "opter": copy.deepcopy(config_2["opter"]),
        # "l2o_dict": l2o_dict_2,
        "l2o_dict": l2o_dict_best_2["best_l2o_dict"],
    },
    "plot_config": {
        "linestyle": "dashed",
    },
    "load_saved": False,
}

In [ ]:
### L2O-CFGD
runs["L2O-CFGD"] = {
    "update_config": {
        "n_iters": update_config_base["n_iters"],
        "optee": update_config_base["optee"],
        "additional_metrics": {
            "alpha": lambda opter, **kwargs: \
                [
                    opter.base_opter.param_groups[pg_i]["alpha"].detach().cpu().numpy()
                    for pg_i in range(len(opter.base_opter.param_groups))
                ] if hasattr(opter, "base_opter") else opter.param_groups[0].get("alpha", None),
            "beta": lambda opter, **kwargs: \
                [
                    opter.base_opter.param_groups[pg_i]["beta"].detach().cpu().numpy()
                    for pg_i in range(len(opter.base_opter.param_groups))
                ] if hasattr(opter, "base_opter") else opter.param_groups[0].get("beta", None),
            "c": lambda opter, **kwargs: \
                [
                    opter.base_opter.param_groups[pg_i]["c"].detach().cpu().numpy()
                    for pg_i in range(len(opter.base_opter.param_groups))
                ] if hasattr(opter, "base_opter") else opter.param_groups[0].get("c", None),
        },
        # "l2o_dict": l2o_dict,
        "l2o_dict": l2o_dict_best["best_l2o_dict"],
    },
    "plot_config": {
        # "color": "orange",
        "linewidth": "1.5",
    },
    "load_saved": False,
}

In [ ]:
### run all
print(f"Running:\n" + ",".join(list(runs.keys())))
for run_name in runs.keys():
    ### prepare run config
    if "l2o_dict" in runs[run_name]["update_config"] \
        and runs[run_name]["update_config"]["l2o_dict"]["opter"].base_opter_cls == L2O_Update:
        run_config = copy.deepcopy(config_2) # L2O, not L2O-CFGD
    else: 
        run_config = copy.deepcopy(config)
    run_config["data"]["data_config"]["device"] = DEVICE
    if "update_config" in runs[run_name] and runs[run_name]["update_config"] is not None:
        run_config.update(runs[run_name]["update_config"])
    print(f"{run_name}:")

    ### remove all non-alphanumeric characters from run_name
    run_name_clean = ''.join(e for e in run_name if e.isalnum())
    run_nickname = f"{run_name_clean}__{dict_to_str(run_config['optee']['optee_config'], ignore=['device'])}__{run_config['data']['data_cls']}__{n_test_runs}runs_{test_run_iters}iters_{test_runs_seed}seed"
    
    ### check if already run    
    # already loaded?
    if "log" in runs[run_name]:
        print("  > Already run.")
        continue

    # already saved?
    if "l2o" in run_name.lower():
        if CKPT_PATH in run_config["ckpt_config"]["ckpt_dir"]:
            save_to = os.path.join(run_config["ckpt_config"]["ckpt_dir_meta_testing"], run_nickname + ".pt")
        else:
            ckpt_dir_local = os.path.basename(run_config["ckpt_config"]["ckpt_dir"])
            save_to = os.path.join(CKPT_PATH, "l2o", ckpt_dir_local, "meta_testing", run_nickname + ".pt")
    else: # baseline
        save_to = os.path.join(CKPT_PATH, "baselines",  run_nickname + ".pt")
    if os.path.exists(save_to) \
        and ("load_saved" not in runs[run_name] or runs[run_name]["load_saved"] == True):
        print(f"  > Already saved. Only loading...\n  > {save_to}")
        runs[run_name] = torch.load(save_to, map_location=torch.device(DEVICE), pickle_module=dill)
        continue

    torch.manual_seed(test_runs_seed)
    np.random.seed(test_runs_seed)

    ### find best lr
    if "lr" in run_config["opter"]["opter_config"] \
        and run_config["opter"]["opter_config"]["lr"] == find_best_lr:
        print("  > Finding best lr...")
        run_config["opter"]["opter_config"]["lr"] = find_best_lr(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            # loss_fn=run_config["loss_fn"],
            n_iters=120,
            n_tests=1,
            consider_metric="loss",
            lrs_to_try=[0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
        )
        print(f"  > Best lr: {run_config['opter']['opter_config']['lr']}")

    print("  > Running...")
    runs[run_name]["log"] = dict()
    for i in range(n_test_runs):
        print(f"    > Run {i+1}/{n_test_runs}...")

        ### check if L2O has been meta-trained
        assert not run_config["opter"]["opter_cls"] == L2O or run_config["l2o_dict"] is not None

        curr_log, optee, _ = do_fit(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            n_iters=run_config["n_iters"],
            l2o_dict=run_config["l2o_dict"],
            in_meta_training=False,
            additional_metrics=run_config["additional_metrics"],
            device=DEVICE,
        )

        for metric_name in curr_log.keys():
            if metric_name not in runs[run_name]["log"]:
                runs[run_name]["log"][metric_name] = []
            runs[run_name]["log"][metric_name].append(curr_log[metric_name])

    runs[run_name]["config"] = run_config

    ### save results
    torch.save(runs[run_name], save_to, pickle_module=dill)

In [ ]:
### plot
plot_metrics(
    baselines={
        k: r for k, r in runs.items() if "L2O-CFGD" not in k
    },
    l2os={
        k: r for k, r in runs.items() if "L2O-CFGD" in k
    },
    metrics=["loss"],
    # show_max_iters=500,
    log_metric=True,
    with_err_bars=True,
    conv_window=10,
    save_fig_to_path=None,
    # y_max=10**1/2,
)

### print sums of losses over individual optimization runs
losses = []
for k, run_log in runs.items():
    if "loss" not in run_log["log"]:
        continue
    losses.append((k, np.array(run_log["log"]["loss"]).sum()))
print("Sum of losses over the optimization run:")
for (k, l) in sorted(losses, key=lambda x: x[1]):
    print("  ", k, "\t", round(l, 3))

## Analyze strategy

In [ ]:
# apply_publication_plt_settings(font_size=16, dpi=600, figsize=(5, 3.5))
# apply_publication_plt_settings(font_size=15, dpi=600, figsize=(13, 8))

In [ ]:
### collect logs
hparams = dict()
for hyperparam_name in ("alpha", "beta", "c"):
    per_param_alphas = []
    for param_idx in range(len(runs["L2O-CFGD"]["log"][hyperparam_name][0][0])):
        per_param_alphas.append([])
        for i in range(len(runs["L2O-CFGD"]["log"][hyperparam_name])):
            per_param_alphas[-1].append([])
            for j in range(len(runs["L2O-CFGD"]["log"][hyperparam_name][i])):
                per_param_alphas[-1][-1].append(
                    runs["L2O-CFGD"]["log"][hyperparam_name][i][j][param_idx]
                )
    for param_idx in range(len(runs["L2O-CFGD"]["log"][hyperparam_name][0][0])):
        per_param_alphas[param_idx] = np.stack(per_param_alphas[param_idx]).reshape(n_test_runs, test_run_iters, -1)
    hparams[hyperparam_name] = per_param_alphas

In [ ]:
### config for plotting
param_idx = 0
test_run_idx = 0
to_plot_name = "c"
to_plot_label = r"$c$"
log_plot = False
max_components = 500
max_iter = test_run_iters
alpha_bg = 0.08
conv_window_mean = 30
fig_dir = "../results/mnist"
fig_dir = None # don't save

### select params
# to_plot_label_cleaned = ''.join(e for e in to_plot_label if e.isalnum())
if param_idx is not None:
    hparams_src = hparams[to_plot_name][param_idx][:,:max_iter]
else:
    hparams_src = np.concatenate(hparams[to_plot_name], axis=2)[:,:max_iter]
component_idxs = np.random.choice(hparams_src.shape[-1], size=min(max_components, hparams_src.shape[-1]), replace=False)
to_plot = hparams_src[test_run_idx,:,component_idxs].T

### where to save the figure
save_fig_to_path = None
if fig_dir is not None:
    fig_name = f"{to_plot_name}_{max_iter}iters_{test_run_idx}run_{param_idx}pidx_{test_runs_seed}seed.pdf"
    save_fig_to_path = os.path.join(fig_dir, fig_name)
print(f"Final destination: {save_fig_to_path if save_fig_to_path is not None else 'None'}")

In [ ]:
### plot
plot_strategy(
    to_plot=to_plot,
    y_label=to_plot_label,
    alpha_bg=alpha_bg,
    conv_window_mean=conv_window_mean,
    save_fig_to_path=save_fig_to_path,
)

In [ ]:
### plot L2O-CFGD's input (grad) to output map
test_run_idx = 0
iters_to_show = [0, 1, 2, 5, 20]

# fig = plt.figure(figsize=(14, 18), facecolor="white")
fig = plt.figure()
# fig.suptitle("L2O + CFGD_ClosedForm")
ax_idx = 1

for i, iter_idx in enumerate(iters_to_show):
    ax = fig.add_subplot(len(iters_to_show), 3, ax_idx)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    
    ### alphas
    sns.scatterplot(
        # x=grads[test_run_idx, iter_idx, component_idxs],
        # y=alphas[test_run_idx, iter_idx, component_idxs],
        x=grads[:, iter_idx, component_idxs],
        y=alphas[:, iter_idx, component_idxs],
        ax=ax,
    )
    if i == len(iters_to_show) - 1:
        ax.set_xlabel(r"$\partial_{x_j} f(x_j)$")
    ax.set_ylabel(r"$\alpha$")
    # ax.set_title(fr"Iteration {iter_idx}")

    ### betas
    ax = fig.add_subplot(len(iters_to_show), 3, ax_idx + 1)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    sns.scatterplot(
        x=grads[test_run_idx, iter_idx, component_idxs],
        y=betas[test_run_idx, iter_idx, component_idxs],
        ax=ax,
    )
    if i == len(iters_to_show) - 1:
        ax.set_xlabel(r"$\partial_{x_j} f(x_j)$")
    ax.set_ylabel(r"$\beta$")
    ax.set_title(fr"Iteration {iter_idx}", pad=10)
    # ax.set_title(fr"$\beta$ (iter {iter_idx})")

    ### cs
    ax = fig.add_subplot(len(iters_to_show), 3, ax_idx + 2)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    sns.scatterplot(
        x=grads[test_run_idx, iter_idx, component_idxs],
        y=cs[test_run_idx, iter_idx, component_idxs],
        ax=ax,
    )
    if i == len(iters_to_show) - 1:
        ax.set_xlabel(r"$\partial_{x_j} f(x_j)$")
    ax.set_ylabel(r"$c$")
    # ax.set_title(fr"$c$ (iter {iter_idx})")

    ax_idx += 3

# fig.tight_layout(h_pad=1.5)
# save_to = os.path.join(
#     config["ckpt_config"]["ckpt_dir"],
#     f"strategy_grad_alpha_beta_c_{test_d}d_{test_m}m_{n_test_runs}runs_{test_run_iters}iters.png"
# )
# fig.savefig(save_to)
plt.tight_layout(h_pad=2.2)
plt.show()

# fig.savefig("../results/strategy.eps", bbox_inches="tight")

In [ ]:
### save stats about the L2O-CFGD strategy
run_name_clean = ''.join(e for e in run_name if e.isalnum())
run_nickname = f"L2OCFGD__{dict_to_str(runs['L2O-CFGD']['config']['optee']['optee_config'])}__{n_test_runs}runs_{test_run_iters}iters_{test_runs_seed}seed"
save_to = os.path.join(runs["L2O-CFGD"]["config"]["ckpt_config"]["ckpt_dir"], run_nickname + "_hparams" + ".pt")

save_to = os.path.join(runs["L2O-CFGD"]["config"]["ckpt_config"]["ckpt_dir"], run_nickname + "_hparams" + ".pt")
torch.save({
    "hyperparams": hparams,
}, save_to, pickle_module=dill)

### Run LCFGD (archived...)

In [ ]:
### collect
alphas = np.stack(hparams["alpha"][0]).reshape(n_test_runs, test_run_iters, -1)
alphas_mean = alphas.mean((0,2))
betas = np.stack(hparams["beta"][0]).reshape(n_test_runs, test_run_iters, -1)
betas_mean = betas.mean((0,2))
cs = np.stack(hparams["c"][0]).reshape(n_test_runs, test_run_iters, -1)  # (n_test_runs, n_iters, D)
cs_mean = cs.mean((0,2))

In [ ]:
alphas_to_set = torch.ones(test_run_iters)
alphas_to_set[:20] = 0.1
alphas_to_set[20:40] = 0.6
alphas_to_set[40:100] = 0.95
alphas_to_set[100:] = 0.99

betas_to_set = torch.zeros(test_run_iters)
betas_to_set[:10] = -0.2
betas_to_set[10:40] = -0.1
betas_to_set[40:100] = 0.15
betas_to_set[100:] = 0.1

cs_to_set = torch.zeros(test_run_iters)
cs_to_set[:10] = -0.5
cs_to_set[10:40] = 0.
cs_to_set[40:] = 0.3

In [ ]:
# lcfgd_run_name = r"LCFGD-AT"
lcfgd_run_name = r"NA-CFGD (test)"
runs[lcfgd_run_name] = dict()
lcfgd_config = {
    "update_config": {
        **update_config_base,
        "opter": {
            "opter_cls": CFGD,
            "opter_config": {
                "lr": 0.1,
                "alpha": 0.98,
                # "alpha": torch.from_numpy(alphas_mean).to(DEVICE),
                # "alpha": alphas_to_set.to(DEVICE),

                # "beta": torch.from_numpy(betas_mean).to(DEVICE),
                "beta": 0.05,
                # "beta": betas_to_set.to(DEVICE),
                # "beta": 0,

                # "c": torch.from_numpy(cs_mean).to(DEVICE),
                # "c": cs_to_set.to(DEVICE),
                # "c": 1,
                # "c": 0.3,
                "c": 0,

                "s": 1,
                "version": "NA",
                "init_points": None,
                # "init_points": [
                #     [torch.randn_like(p, requires_grad=False, device=DEVICE)] for _, p in _tmp_optee.all_named_parameters()
                # ],
                "device": DEVICE,
            },
        },
    },
    "plot_config": {
        "linestyle": "dashed",
    },
}

In [ ]:
### run
runs[lcfgd_run_name]["config"] = copy.deepcopy(config)
runs[lcfgd_run_name]["config"].update(lcfgd_config["update_config"])
runs[lcfgd_run_name]["config"]["data"]["data_config"]["device"] = DEVICE
runs[lcfgd_run_name]["plot_config"] = lcfgd_config["plot_config"]
runs[lcfgd_run_name]["config"]["device"] = DEVICE

lcfgd_log = do_fit(
    opter_cls=runs[lcfgd_run_name]["config"]["opter"]["opter_cls"],
    opter_config=runs[lcfgd_run_name]["config"]["opter"]["opter_config"],
    optee_cls=runs[lcfgd_run_name]["config"]["optee"]["optee_cls"],
    optee_config=runs[lcfgd_run_name]["config"]["optee"]["optee_config"],
    data_cls=runs[lcfgd_run_name]["config"]["data"]["data_cls"],
    data_config=runs[lcfgd_run_name]["config"]["data"]["data_config"],
    n_iters=runs[lcfgd_run_name]["config"]["n_iters"],
    l2o_dict=runs[lcfgd_run_name]["config"]["l2o_dict"],
    in_meta_training=False,
    additional_metrics=runs[lcfgd_run_name]["config"]["additional_metrics"],
    device=DEVICE,
)[0]

### append log
runs[lcfgd_run_name]["log"] = dict()
for metric_name in lcfgd_log.keys():
    if metric_name not in runs[lcfgd_run_name]["log"]:
        runs[lcfgd_run_name]["log"][metric_name] = []
    runs[lcfgd_run_name]["log"][metric_name].append(lcfgd_log[metric_name])

In [ ]:
plot_metric(
    baselines={
        k: r for k, r in runs.items()
        if "L2O-CFGD" not in k and "LCFGD" not in k
    },
    l2os={
        k: r for k, r in runs.items()
        if "L2O-CFGD" in k or "LCFGD" in k
    },
    metric="loss",
    show_max_iters=1000,
    log_metric=True,
    with_err_bars=True,
    conv_window=10,
    save_fig_to_path=None,
)

In [ ]:
### reorder runs
# runs = {
#     "GD": runs["GD"],
#     "L2O-CFGD": runs["L2O-CFGD"],
# }

### clear runs
ks = list(runs.keys())
for k in ks:
    if "lcfgd" in k.lower():
        del runs[k]